# Hadoop Streaming assignment 3: Name Count
Make WordCount program for all the names in the dataset. Name is a word with the following properties:

* The first character is not a digit (other characters can be digits).
* The first character is uppercase, all the other characters that are letters are lowercase.
* There are less than 0.5% occurrences of this word, when this word regardless to its case appears in the dataset and the condition (2) is not met.

Order by quantity, most popular first, output format:

<code>name <tab> count</code>

The result is the 5th line in the output.

The result on the sample dataset:

<code>french 5742</code>

If you want to deploy the environment on your own machine, please use bigdatateam/yarn-notebook Docker container.

In [3]:
%%writefile test.dat

1	For The Horde!
2	for the horde
3	For$ #THe HoRde!!!
42	A A A A A A A A rare Iam the the
1	For For for for
2	1for 1The 1Horde!
3	Horde Horde Horde!

Overwriting test.dat


cat test.dat

In [4]:
%%writefile mapper1.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

total = 0

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        total += 1       
        cond1 = word[0].isalpha()
        cond2 = not word[0].islower() and word[1:].islower()        
        if (cond1):
            print "%s\t%d\t%d" % (word, 1, cond2)
        
print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % total

Overwriting mapper1.py


In [5]:
cat test.dat | python2 ./mapper1.py | sort

reporter:counter:Wiki stats,Total words,31
A	1	0
A	1	0
A	1	0
A	1	0
A	1	0
A	1	0
A	1	0
A	1	0
for	1	0
for	1	0
for	1	0
For	1	1
For	1	1
For	1	1
For	1	1
horde	1	0
HoRde	1	0
Horde	1	1
Horde	1	1
Horde	1	1
Horde	1	1
Iam	1	1
rare	1	0
the	1	0
the	1	0
the	1	0
THe	1	0
The	1	1


In [26]:
%%writefile reducer1.py

import sys

current_key=None
current_group = []

for line in sys.stdin:
    try:
        key, count, cond2 = line.strip().split('\t', 2)
        count = int(count)
        cond2 = int(cond2)        
    except ValueError as e:
        continue
    
    if current_key != key: # the next case sensitive word 
        if current_key:
            current_group.append([current_key, key_sum, current_cond2])
            
            if current_key.lower() != key.lower(): # next not case sensitive word
                def print_group(arr):
                    word_sum = 0
                    for i in arr:
                        word_sum += i[1]
                    for i in arr:
                        print "%s\t%d\t%d\t%d" % (i[0], i[1], i[2], word_sum)                
                
                print_group(current_group)
                current_group = []
        
        current_key = key
        key_sum = 0
        current_cond2 = cond2    
    key_sum += count

if current_key:
    print_group(current_group)

Overwriting reducer1.py


In [27]:
cat test.dat | python2 ./mapper1.py | sort | python2 ./reducer1.py

reporter:counter:Wiki stats,Total words,31
A	8	0	8
for	3	0	7
For	4	1	7
horde	1	0	6
HoRde	1	0	6
Horde	4	1	6
Iam	1	1	1
rare	1	0	1
the	3	0	4
THe	1	0	4
